In [1]:
# DAY 01
nums = [int(x) for x in open("../data/01.txt").readlines()]

def sliding_window(arr, n):
    for i in range(0, len(arr) - n + 1):
        yield arr[i:i+n]

res1 = sum(x[1] > x[0] for x in sliding_window(nums, 2))
res2 = sum(x[1] > x[0] for x in sliding_window(
    [sum(y) for y in sliding_window(nums, 3)], 
    2))

print(f'Answer 1: {res1}\nAnswer 2: {res2}')

Answer 1: 1557
Answer 2: 1608


In [2]:
# DAY 02
from functools import reduce

commands = [(x.split()[0], int(x.split()[1])) 
             for x in open("../data/02.txt").readlines()]

def step1(pos, command):
    (x, depth) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth - offs)
    elif cmd == 'down':
        return (x, depth + offs)
    elif cmd == 'forward':
        return (x + offs, depth)
    
def step2(pos, command):
    (x, depth, aim) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth, aim - offs)
    elif cmd == 'down':
        return (x, depth, aim + offs)
    elif cmd == 'forward':
        return (x + offs, depth + aim * offs, aim)
        
pos1 = reduce(step1, commands, (0, 0))
pos2 = reduce(step2, commands, (0, 0, 0))
print(f'Answer 1: {pos1[0] * pos1[1]}\nAnswer 2: {pos2[0] * pos2[1]}')

Answer 1: 1728414
Answer 2: 1765720035


In [3]:
# DAY 03
def get_most_least_common(nums, pos):
    n0 = 0
    n1 = 0
    for num in nums:
        n0 += num[pos] == '0'
        n1 += num[pos] == '1'
    return ('1', '0') if n1 >= n0 else ('0', '1')

def part1(nums):
    lc, mc = "", ""
    for i in range(0, len(nums[0])):
        (m, l) = get_most_least_common(nums, i)
        mc += m
        lc += l
    return int(mc, 2) * int(lc, 2)
     
def part2(nums):
    mnums = nums[:]
    lnums = nums[:]
    for i in range(0, len(nums[0])):
        if len(mnums) > 1:
            (m, l) = get_most_least_common(mnums, i)
            mnums = [x for x in mnums if x[i] == m]

        if len(lnums) > 1:
            (m, l) = get_most_least_common(lnums, i)
            lnums = [x for x in lnums if x[i] == l]

    return int(mnums[0], 2) * int(lnums[0], 2)

nums = [x.strip() for x in open('../data/03.txt').readlines()]
print(f'Answer 1: {part1(nums)}\nAnswer 2: {part2(nums)}')

Answer 1: 2972336
Answer 2: 3368358


In [96]:
# DAY 04
def get_boards(data):
    cells = []
    for line in data:
        if line == '':
            yield cells
            cells = []
        else:
            cells.append([int(x) for x in line.split()])
    yield cells

def is_full(board, pos, is_vert):
    for i in range(len(board)):
        n = board[i][pos] if is_vert else board[pos][i]
        if n != -1:
            return False
    return True

def has_winner(board):
    return any(map(lambda i: 
                       is_full(board, i, True) or is_full(board, i, False), 
                   range(len(board))))

def apply_num(board, num):
    for row in board:
        for i, el in enumerate(row):
            if el == num:
                row[i] = -1
            
def run_simulation(nums, boards):
    winners, scores = [], []
    for n in nums:
        for i, board in enumerate(boards):
            if i in winners:
                continue
            apply_num(board, n)
            if has_winner(board):
                s = n * sum(x for row in board for x in row if x != -1)
                winners.append(i)
                scores.append(s)
    return scores
             
lines = [x.strip() for x in open('../data/04.txt').readlines()]
nums = [int(x) for x in lines[0].split(",")]
boards = list(get_boards(lines[2:]))
scores = run_simulation(nums, boards)

print(f"Answer 1: {scores[0]}\nAnswer 2: {scores[-1]}")

Answer 1: 44736
Answer 2: 1827


In [9]:
# DAY 05
from collections import defaultdict

def sgn(x):
    if x == 0:
        return 0
    return 1 if x > 0 else -1

def stroke(a, b, ptmap, skip_diagonals):
    (x1, y1) = a
    (x2, y2) = b
    dx = sgn(x2 - x1)
    dy = sgn(y2 - y1)
    
    if skip_diagonals and dx != 0 and dy != 0:
        return
    
    cx, cy = x1, y1
    while True:
        ptmap[(cx, cy)] += 1
        if cx == x2 and cy == y2:
            break
        cx += dx
        cy += dy
        
def find_num_overlaps(points, skip_diagonals):
    ptmap = defaultdict(int)
    for a, b in points:
        stroke(a, b, ptmap, skip_diagonals)
    return sum(ptmap[x] >= 2 for x in ptmap)

def parse_pt(pt):
    (x, y) = [int(c) for c in pt.split(',')]
    return (x, y)

points = [list(map(parse_pt, x.strip().split(' -> ')))
          for x in open('../data/05.txt').readlines()]
res1 = find_num_overlaps(points, True)
res2 = find_num_overlaps(points, False)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 7436
Answer 2: 21104


In [387]:
# DAY 06
def sum_spawned(nums, total_days):
    spawned = {}

    def get_n_spawned(n, days):
        if days <= n:
            return 0
        if (n, days) in spawned:
            return spawned[(n, days)]
        res = (days - n - 1) // 7 + 1

        for i in range(res):
            res += get_n_spawned(8, days - n - i * 7 - 1)
        spawned[(n, days)] = res
        return res

    return sum(get_n_spawned(x, total_days) for x in nums) + len(nums)

nums = [int(x) for x in open('../data/06.txt').read().split(',')]
print(f"Answer 1: {sum_spawned(nums, 80)}\nAnswer 2: {sum_spawned(nums, 256)}")

Answer 1: 386755
Answer 2: 1732731810807


In [19]:
# DAY 07
import statistics

def part1(nums):
    c = statistics.median(nums)
    return sum(abs(x - c) for x in nums)

# simple gradient descent
def fmin(p0, f, num_it=1000):
    EPS = 0.001
    p = p0
    for i in range(num_it):
        pr = f(p + EPS)
        pl = f(p - EPS)
        dp = (pr - pl)/(2 * EPS)
        p = p - dp * EPS
    return p

def part2(nums):
    def f(x):
        res = 0
        for n in nums:
            d = abs(x - n)
            res += d * (d + 1)/2
        return res
    minx = int(round(fmin(statistics.median(nums), f)))
    return f(minx)

nums = [int(x) for x in open('../data/07.txt').read().split(',')]
print(f"Answer 1: {int(part1(nums))}\nAnswer 2: {int(part2(nums))}")

Answer 1: 343468
Answer 2: 96086265


In [ ]:
# DAY08
CHARS = 'abcdefg'
DIGITS = ['abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf',
         'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg']
DIGITS_MAP = {x: i for i, x in enumerate(DIGITS)}
NUM_SEGMENTS = 7

def normalize_mapping(mapping):
    changed = True
    while changed:
        changed = False
        for i, c in enumerate(mapping):
            if sum(c) == 1:
                p = c.index(1)
                for j in range(NUM_SEGMENTS):
                    if j != i and mapping[j][p] == 1:
                        mapping[j][p] = 0
                        changed = True
                        
def is_valid_mapping(mapping):
    if not all(sum(c) == 1 for c in mapping):
        return False
    for i in range(NUM_SEGMENTS):
        s = [mapping[j][i] for j in range(NUM_SEGMENTS)]
        if sum(s) != 1:
            return False
    return True

def get_mapping(mapping, combs, pos):
    print(f"mapping: {mapping} pos: {pos}")
    if pos == len(combs):
        normalize_mapping(mapping)
        return mapping if is_valid_mapping(mapping) else None
    comb = combs[pos]
    for digit in DIGITS:
        if len(digit) != len(comb):
            continue
        print(digit, comb)
        mapping1 = [x[:] for x in mapping]
        for d in [x for x in CHARS if x not in digit]: 
            for c in comb:
                mapping1[ord(c) - ord('a')][ord(d) - ord('a')] = 0
        res_mapping = get_mapping(mapping1, combs, pos + 1)
        if res_mapping != None:
            return res_mapping
    return None
  
def decode_digit(m, combs):
    res = 0
    for comb in combs:
        s = "".join(sorted(chr(m[ord(c) - ord('a')].index(1) + ord('a')) for c in comb))
        if s not in DIGITS_MAP:
            return None
        digit = DIGITS_MAP[s]
        res = res * 10 + digit
    return res

def decode_sum(inp):
    res = 0
    for combs, to_decode in inp:
        mapping = [[1] * NUM_SEGMENTS] * NUM_SEGMENTS
        combs.sort(key=len)
        mapping = get_mapping(mapping, combs, 0)
        res += decode_digit(mapping, to_decode)
    return res

def parse(line):
    return line.strip().split(" ")

inp = [list(map(parse, x.strip().split("|"))) 
       for x in open('../data/08.txt').readlines()]

res1 = sum(len(x) in [2, 3, 4, 7] for a, b in inp for x in b)
res2 = decode_sum(inp)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

In [315]:
# DAY09
import math

OFFS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def find_minima(board):
    res = []
    for j, row in enumerate(board):
        for i, h in enumerate(row):
            is_min = True
            for dx, dy in OFFS:
                x, y = i + dx, j + dy
                if x >= 0 and x < len(row) and y >= 0 and y < len(board) and board[y][x] <= h:
                    is_min = False
                    break
            if is_min:
                res.append((i, j))
    return res
    
def fill_cell(board, bidx, pos, basin_idx):
    x, y = pos
    if x < 0 or x >= len(board[0]) or y < 0 or y >= len(board):
        return
    b = board[y][x]
    if b == 9:
        bidx[y][x] = 0
        return
        
    if bidx[y][x] >= 0:
        return
    bidx[y][x] = basin_idx
    h = board[y][x]
    for dx, dy in OFFS:
        x1, y1 = x + dx, y + dy
        fill_cell(board, bidx, (x1, y1), basin_idx)

def find_basins(board):
    w, h = len(board[0]), len(board)
    bidx = [[-1]*w for i in range(h)]
    num_basins = 0
    for j in range(h):
        for i in range(w):
            if bidx[j][i] == -1:
                fill_cell(board, bidx, (i, j), num_basins + 1)
                num_basins += 1
    return bidx
                
board = [list(map(int, list(x.strip()))) for x in open('../data/09.txt').readlines()]

mins = find_minima(board)
res1 = sum(map(lambda p: board[p[1]][p[0]] + 1, mins))

basins = find_basins(board)
max_basin = max(map(max, basins))

basin_sizes = []
for bidx in range(1, max_basin + 1):
    size = 0
    w, h = len(board[0]), len(board)
    for j in range(h):
        for i in range(w):
            if basins[j][i] == bidx:
                size += 1
    basin_sizes.append(size)

res2 = math.prod(sorted(basin_sizes, reverse=True)[:3])
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 588
Answer 2: 964712


In [3]:
# DAY10
CMAP = {"[": "]", "{": "}", "<": ">", "(": ")"}
SCORES1 = {")": 3, "]": 57, "}": 1197, ">": 25137}
SCORES2 = {")": 1, "]": 2, "}": 3, ">": 4}
CORRUPTED = 1
INCOMPLETE = 2

def find_match(line):
    chars = []
    for c in line:
        if c in "[({<":
            chars.append(c)
        else:
            current = chars.pop()
            if c != CMAP[current]:
                return (CORRUPTED, c)
    return (INCOMPLETE, "".join(CMAP[x] for x in chars[::-1]))

def score2(line):
    res = 0
    for c in line:
        res = res * 5 + SCORES2[c]
    return res

lines = [x.strip() for x in open('../data/10.txt').readlines()]
matches = [find_match(line) for line in lines]
res1 = sum(SCORES1[x] for (t, x) in matches if t == CORRUPTED)
scores2 = sorted(score2(x) for (t, x) in matches if t == INCOMPLETE)
res2 = scores2[len(scores2) // 2]
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 216297
Answer 2: 2165057169


In [265]:
# DAY11

OFFS = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)] 

def flash(board, x, y):
    res = 1
    board[y][x] == 11
    for dx, dy in OFFS:
        cx, cy = x + dx, y + dy
        if cx >= 0 and cx < len(board[0]) and cy >= 0 and cy < len(board):
            board[cy][cx] += 1            
            if board[cy][cx] == 10:
                res += flash(board, cx, cy)
    return res

def step(board):
    num_flashes = 0
    for j in range(len(board)):
        for i in range(len(board[0])):
            board[j][i] += 1
            if board[j][i] == 10:
                num_flashes += flash(board, i, j)    
    for j in range(len(board)):
        for i in range(len(board[0])): 
            if board[j][i] >= 10:
                board[j][i] = 0
    return num_flashes
    
board = [list(map(int, list(s))) for s in open("../data/11.txt").read().split()]

res1 = 0
board1 = [row[:] for row in board]
for i in range(100):
    res1 += step(board1)

res2 = 0
while True:
    num_flashes = step(board)
    res2 += 1
    if num_flashes == len(board) * len(board[0]):
        break

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 1683
Answer 2: 788


In [314]:
# DAY12
from collections import defaultdict

def traverse1(graph, pos, visited):    
    res = 0
    for next_pos in graph[pos]:
        if next_pos == "end":
            res += 1
            continue
        if next_pos == "start":
            continue
        if next_pos.islower() and visited[next_pos] == 1:
            continue
        visited[next_pos] += 1
        res += traverse1(graph, next_pos, visited)
        visited[next_pos] -= 1
    return res
        
def traverse2(graph, pos, visited, visited_small=None):    
    res = 0
    for next_pos in graph[pos]:
        if next_pos == "end":
            res += 1
            continue
        if next_pos == "start":
            continue
        if next_pos.islower():
            if visited[next_pos] == 1:
                if visited_small == None or visited_small == next_pos:
                    visited[next_pos] += 1
                    res += traverse2(graph, next_pos, visited, next_pos)
                    visited[next_pos] -= 1
                continue
            if visited[next_pos] == 2:
                continue
        visited[next_pos] += 1
        res += traverse2(graph, next_pos, visited, visited_small)
        visited[next_pos] -= 1
    return res

edges = [line.strip().split("-") for line in open("../data/12.txt").readlines()]

verts = set()
for a, b in edges:
    verts.add(a)
    verts.add(b)
verts = list(verts)
    
graph = defaultdict(list)
for a, b in edges:
    graph[a].append(b)
    graph[b].append(a)
    
visited = {v: 0 for v in graph}
res1 = traverse1(graph, "start", visited)
res2 = traverse2(graph, "start", visited)

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 4970
Answer 2: 137948


In [110]:
# DAY13
def fold(coords, dir, pos):
    res = []
    if dir == 'x':
        for x, y in coords:
            if x < pos:
                res.append((x, y))
            else:
                res.append((2*pos - x, y))
    else:
        for x, y in coords:
            if y < pos:
                res.append((x, y))
            else:
                res.append((x, 2*pos - y))
    return res
    
clines, flines = open('../data/13.txt').read().split("\n\n")
coords = [list(map(int, x.split(","))) for x in clines.split("\n")]
fdirs = [x[11:].split("=") for x in flines.strip().split("\n")]

coords = set(fold(coords, fdirs[0][0], int(fdirs[0][1])))
res1 = len(set(coords))

for dir, pos in fdirs[1:]:
    coords = set(fold(coords, dir, int(pos)))

mx, my = max(coords)
m = [['.']*(mx + 1) for i in range(my + 1)] 
for x, y in coords:
    m[y][x] = "#"
res2 = "\n".join("".join(x) for x in m)
print(f"Answer 1: {res1}\nAnswer 2:\n{res2}")

Answer 1: 708
Answer 2:
####.###..#....#..#.###..###..####.#..#
#....#..#.#....#..#.#..#.#..#.#....#..#
###..###..#....#..#.###..#..#.###..####
#....#..#.#....#..#.#..#.###..#....#..#
#....#..#.#....#..#.#..#.#.#..#....#..#
####.###..####..##..###..#..#.#....#..#


In [232]:
# DAY14
from collections import defaultdict

def step(hist, rules):
    res = defaultdict(int)
    for s, cnt in hist.items():
        if s in rules:
            c = rules[s]
            res[s[0] + c] += cnt
            res[c + s[1]] += cnt
        else:
            res[s] = cnt
    return res

def get_pairs_hist(pattern):
    res = defaultdict(int)
    for i in range(1, len(pattern)):
        res[pattern[i - 1: i + 1]] += 1
    return res

def eval_pattern(start_pattern, rules, steps):
    hist = get_pairs_hist(start_pattern)
    for i in range(steps):
        hist = step(hist, rules)
        
    char_hist = defaultdict(int)
    for s, cnt in hist.items():
        char_hist[s[0]] += cnt
    char_hist[start_pattern[-1]] += 1

    counts = sorted((char_hist[c], c) for c in char_hist.keys())
    return counts[-1][0] - counts[0][0]
    
lines = open('../data/14.txt').readlines()
start_pattern = lines[0].strip()
rules = {x[0]: x[1] for x in map(lambda s: s.strip().split(" -> "), lines[2:])}

res1 = eval_pattern(start_pattern, rules, 10)
res2 = eval_pattern(start_pattern, rules, 40)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 2621
Answer 2: 2843834241366


In [571]:
# DAY15
import heapq

def get_risk(board, x, y):
    w, h = len(board[0]), len(board)
    res = board[y % h][x % w]
    res += x // w + y // h
    if res > 9:
        res = res % 10 + 1
    return res

def get_min_risk(board, pos, goal):
    w, h = len(board[0]), len(board)
    to_explore = [(0, pos)]
    scores = {pos: 0}
    
    while len(to_explore) > 0:
        score, pos = heapq.heappop(to_explore)
        x, y = pos
        if x == goal[0] and y == goal[1]:
            return scores[pos]

        for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            cx, cy = x + dx, y + dy
            if cx < 0 or cy < 0 or cx > goal[0] or cy > goal[1]:
                continue
            cpos = (cx, cy)
            cscore = score + get_risk(board, cx, cy)
            if cpos not in scores or scores[cpos] > cscore:
                scores[cpos] = cscore
                heapq.heappush(to_explore, (cscore, cpos))
    
board = [list(map(int, list(s))) for s in open("../data/15.txt").read().split()]
w, h = len(board[0]), len(board)
res1 = get_min_risk(board, (0, 0), (w - 1, h - 1))
res2 = get_min_risk(board, (0, 0), (w * 5 - 1, h * 5 - 1))
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 527
Answer 2: 2887


In [1]:
# DAY16
LITERAL_TYPE = 4

def to_bin(data):
    s = bin(int(data, 16))[2:]
    return '0' * (len(data) * 4 - len(s)) + s

def decode(packet, pos=0):
    v = int(packet[pos:pos + 3], 2)
    t = int(packet[pos + 3:pos + 6], 2)
    pos += 6
    if t == LITERAL_TYPE:
        i = 0
        res = ''
        while True:
            res += packet[pos + i + 1 : pos + i + 5]
            if packet[pos + i] == '0':
                break
            i += 5
        return pos + i + 5, (v, t, int(res, 2))
    else:
        length_id = packet[pos]
        pos += 1
        subpackets = []
        if length_id == '0':
            length_bits = int(packet[pos : pos + 15], 2)
            pos += 15
            start_pos = pos
            while pos - start_pos < length_bits:
                pos, subpacket = decode(packet, pos)
                subpackets.append(subpacket)
        else:
            num_sub_packets = int(packet[pos : pos + 11], 2)
            pos += 11
            for i in range(num_sub_packets):
                pos, subpacket = decode(packet, pos)
                subpackets.append(subpacket)
        return pos, (v, t, subpackets)
    
def get_version_sum(decoded_packet):
    v, _, sub = decoded_packet
    res = v
    if isinstance(sub, list):
        for s in sub:
            res += get_version_sum(s)
    return res

def eval_packet(decoded_packet):
    v, t, sub = decoded_packet
    if t == 0:
        return sum(eval_packet(p) for p in sub)
    elif t == 1:
        res = 1
        for p in sub:
            res *= eval_packet(p)
        return res
    elif t == 2:
        return min(eval_packet(p) for p in sub)
    elif t == 3:
        return max(eval_packet(p) for p in sub)
    elif t == 5:
        return int(eval_packet(sub[0]) > eval_packet(sub[1]))
    elif t == 6:
        return int(eval_packet(sub[0]) < eval_packet(sub[1]))
    elif t == 7:
        return int(eval_packet(sub[0]) == eval_packet(sub[1]))
    else:
        return sub

data = open("../data/16.txt").read().strip()
packet = to_bin(data)
_, dp = decode(packet)
res1 = get_version_sum(dp)
res2 = eval_packet(dp)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 977
Answer 2: 101501020883


In [775]:
# DAY17
import math

def parse_ext(data):
    parts = (data[len("target area: "):]).split(", ")
    for p in parts:
        v1, v2 = p[2:].split("..")
        yield (int(v1), int(v2))
        
def in_ext(ext, x, y):
    (x1, x2), (y1, y2) = ext
    return x >= x1 and x <= x2 and y >= y1 and y <= y2

def simulate(ext, vx, vy):
    x, y = 0, 0
    maxh = 0
    while True:
        maxh = max(y, maxh)
        if in_ext(ext, x, y):
            return maxh
        elif x > ext[0][1] or y < ext[1][0]:
            return None
        x += vx
        y += vy
        vx = max(0, vx - 1)
        vy -= 1
        
line = open("../data/17.txt").read().strip()
ext = list(parse_ext(line))

maxh = 0
numv = 0

# solve (n + 1) * n / 2 = ext.minx to find minimum vx
min_vx = (int(math.sqrt(8 * ext[0][0])) - 1) // 2 + 1
max_vx = ext[0][1]
min_vy = -5 * min_vx
max_vy = abs(ext[1][0])
for vx in range(min_vx, max_vx + 1):
    for vy in range(min_vy, max_vy + 1):
        h = simulate(ext, vx, vy)
        if h != None:
            maxh = max(maxh, h)
            numv += 1
            
print(f"Answer 1: {maxh}\nAnswer 2: {numv}")

Answer 1: 3655
Answer 2: 1447


In [118]:
# DAY18
import json
import copy

OP_NONE = 0
OP_SPLIT = 1
OP_EXPLODE = 2
OP_SPLIT_EXPLODE = 3
MAX_DEPTH = 5

def parse_num(line):
   return json.loads(line) 
        
def reduce(num, can_split=True):
    prev_regular = None
    next_regular_inc = 0
    current_op = OP_NONE
    
    def rec(num, depth=0):
        nonlocal prev_regular, next_regular_inc, current_op
        for i, n in enumerate(num):
            if current_op != OP_NONE:
                if not isinstance(n, list):
                    num[i] += next_regular_inc
                    next_regular_inc = 0
                    return
                else:
                    rec(n, depth + 1)
            elif isinstance(n, list):
                if depth >= 3:
                    if prev_regular != None:
                        prev_regular[0][prev_regular[1]] += n[0]
                    next_regular_inc = n[1]
                    num[i] = 0
                    current_op = OP_EXPLODE
                else:
                    rec(n, depth + 1)
            else:
                if can_split and n >= 10:
                    if depth >= 3:
                        if prev_regular != None:
                            prev_regular[0][prev_regular[1]] += n // 2
                        next_regular_inc = (n + 1) // 2
                        num[i] = 0
                        current_op = OP_SPLIT_EXPLODE
                        continue
                    else:
                        num[i] = [n // 2, (n + 1) // 2]
                        current_op = OP_SPLIT
                        return
                prev_regular = (num, i)

    rec(num, 0)
    return current_op

def get_magnitude(num):
    if not isinstance(num, list):
        return num
    return 3 * get_magnitude(num[0]) + 2 * get_magnitude(num[1])

def eval_nums(nums):
    res = nums[0]
    for i in range(1, len(nums)):
        res = [res, nums[i]]
        while True:
            op = reduce(res, False)
            if op == OP_EXPLODE:
                continue
            op = reduce(res, True)
            if op == OP_NONE:
                break
    return res
    
lines = open('../data/18.txt').readlines()
nums = [parse_num(line) for line in lines]

nums1 = copy.deepcopy(nums)
num1 = eval_nums(nums1)
res1 = get_magnitude(num1)

res2 = 0
for i in range(len(nums)):
    for j in range(0, len(nums)):
        if i == j:
            continue
        num = eval_nums([copy.deepcopy(nums[i]), copy.deepcopy(nums[j])])
        mag = get_magnitude(num)
        if res2 < mag:
            res2 = mag

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 3725
Answer 2: 4832


In [116]:
# DAY19
import numpy as np
import math

SCANNER_DIST = 1000
MIN_POINT_MATCHES = 12
MIN_DIR_MATCHES = math.factorial(MIN_POINT_MATCHES) // 2 // math.factorial(MIN_POINT_MATCHES - 2)

def enum_transforms():
    for i in range(3):
        for sign1 in [1, -1]:
            for j in range(3):
                if i == j:
                    continue
                for sign2 in [1, -1]:
                    tm = np.zeros((3, 3))
                    tm[0][i] = sign1
                    tm[1][j] = sign2
                    tm[2] = np.cross(tm[0], tm[1])
                    yield tm
                    
TRANSFORMS = list(enum_transforms())

def get_offset(coords1, coords2, dirs1, dirs2):    
    common_dirs = set(dirs1.keys()).intersection(set(dirs2.keys()))
    if len(common_dirs) < MIN_DIR_MATCHES:
        return []
    
    matching_dir = next(iter(common_dirs))
    c1 = coords1[dirs1[matching_dir][0]]    
    pts1 = set(tuple(p) for p in coords1)

    for i in [0, 1]:
        c2 = coords2[dirs2[matching_dir][i]]
        offs = np.subtract(c2, c1)
        pts2 = set(tuple(np.subtract(c, offs)) for c in coords2)
        if len(pts2.intersection(pts1)) >= MIN_POINT_MATCHES:
            return offs
    return []

def get_scanner_positions(data):
    res = [np.zeros(3) for _ in range(len(data))]
    found = set([0])
    to_find = set(range(1, len(data))) 
    non_overlapping = set()
    
    def compute_dirs(coords):
        return {tuple(np.abs(np.subtract(p2, p1))): 
                (i, j) for i, p1 in enumerate(coords) 
                for j, p2 in enumerate(coords)}
    
    # precompute both transformed coordinates and directions
    dirs = {}
    coords = {}
    for i in range(len(data)):
        for k, tm in enumerate(TRANSFORMS):
            coords_tm = [np.dot(tm, c) for c in data[i]]
            coords[(i, k)] = coords_tm
            dirs[(i, k)] = compute_dirs(coords_tm)
                
    def test_pair(i, j):
        nonlocal found, to_find, res, coords, dirs, data
        if (i, j) in non_overlapping:
            return False
        for k, tm in enumerate(TRANSFORMS):
            offs = get_offset(data[j], coords[(i, k)], dirs[(j, 0)], dirs[(i, k)])
            if len(offs) == 0:
                continue
            data[i] = [np.subtract(np.dot(tm, pt), offs) for pt in coords[(i, 0)]]
            dirs[(i, 0)] = compute_dirs(data[i])
            found.add(i)
            to_find.remove(i)
            res[i] = -offs
            return True
        non_overlapping.add((i, j))
        return False
    
    while len(to_find) > 0:
        pairs = ((i, j) for i in to_find for j in found)
        for (i, j) in pairs:
            if test_pair(i, j):
                break
    return res

def parse(data):
    lines = data.strip().split("\n")
    coords = [list(map(int, s.strip().split(","))) for s in lines[1:]]
    return coords

def manhattan_dist(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])

data = [parse(x) for x in open('../data/19.txt').read().split("\n\n")]

scanner_pos = [tuple(x) for x in get_scanner_positions(data)]
res1 = len(set(tuple(x) for y in data for x in y))

res2 = 0
for i in range(0, len(scanner_pos)):
    for j in range(i + 1, len(scanner_pos)):
        res2 = max(res2, manhattan_dist(scanner_pos[i], scanner_pos[j]))
print(f"Answer 1: {res1}\nAnswer 2: {int(res2)}")

Answer 1: 385
Answer 2: 10707


In [30]:
# DAY20
def is_need_invert(lookup):
    return lookup[0] == 1 and lookup[-1] == 0

def get_set_ext(pixels):
    minx, miny = 0, 0
    maxx, maxy = 0, 0
    for x, y in pixels:
        minx = min(minx, x)
        maxx = max(maxx, x)
        miny = min(miny, y)
        maxy = max(maxy, y)
    return ((minx, maxx), (miny, maxy))

def step(pixels, lookup, is_lit_set=True):
    need_invert = is_need_invert(lookup)
    lit_val = 1 if need_invert ^ is_lit_set else 0
    (minx, maxx), (miny, maxy) = get_set_ext(pixels)
    res = set()
    for j in range(miny - 1, maxy + 2):
        for i in range(minx - 1, maxx + 2):
            mask = 0
            for dj in [-1, 0, 1]:
                for di in [-1, 0, 1]:
                    x, y = i + di, j + dj
                    bit = int((x, y) in pixels)
                    if not is_lit_set:
                        bit = 1 - bit
                    mask = mask * 2 + bit
            val = lookup[mask]
            if val == lit_val:
                res.add((i, j))

    return res

def eval_image(pixels, lookup, steps):
    need_invert = is_need_invert(lookup)
    for i in range(steps):
        is_lit_set = not need_invert or (i % 2 == 0)
        pixels = step(pixels, lookup, is_lit_set)
    return len(pixels)
        
lines = [x.strip() for x in open('../data/20.txt').readlines()]
lookup = [int(c == '#') for c in list(lines[0])]
pixels = set((i, j) 
             for j, line in enumerate(lines[2:]) 
             for i, c in enumerate(line)
             if c == '#')
            
res1 = eval_image(pixels, lookup, 2)
res2 = eval_image(pixels, lookup, 50)

print(f"Answer 1: {res1}\nAnswer 2: {int(res2)}")

Answer 1: 5044
Answer 2: 18074


In [4]:
# DAY21
def parse(line):
    parts = line.split(" ")
    return int(parts[4])
        
def game1(start_pos):
    def roll():
        cur = 1
        while True:
            yield cur + (cur + 1) + (cur + 2)
            cur += 3
        
    scores = [0, 0]
    cur_pos = start_pos[:]
    cur_player = 0
    num_rolls = 0
    roll_gen = roll()
    while scores[0] < 1000 and scores[1] < 1000:
        steps = next(roll_gen)
        num_rolls += 1
        pos = cur_pos[cur_player] + steps
        pos = (pos - 1) % 10 + 1
        cur_pos[cur_player] = pos
        scores[cur_player] += pos
        if scores[cur_player] >= 1000:
            break
        cur_player = 1 - cur_player
    return min(s * num_rolls * 3 for s in scores)

def game2(start_pos):
    mem = {}
    def get_num_winning(cur_pos, scores, cur_player):
        nonlocal mem
        key = (cur_pos, scores, cur_player)
        if key in mem:
            return mem[key]
        if scores[0] >= 21 or scores[1] >= 21:
            if scores[0] > scores[1]:
                return (1, 0)
            else:
                return (0, 1)
        res = [0, 0]
        for i in range(1, 4):
            for j in range (1, 4):
                for k in range (1, 4):
                    steps = i + j + k
                    pos = cur_pos[cur_player] + steps
                    pos = (pos - 1) % 10 + 1
                    wins = get_num_winning((pos, cur_pos[1]), 
                                            (scores[0] + pos, scores[1]), 
                                            1) if cur_player == 0 else \
                    get_num_winning((cur_pos[0], pos), 
                                            (scores[0], scores[1] + pos), 
                                            0)
                    res[0] += wins[0]
                    res[1] += wins[1]
                                               
        mem[key] = res
        return res
    return get_num_winning(tuple(start_pos), (0, 0), 0)
  
positions = [parse(x.strip()) for x in open('../data/21t1.txt').readlines()]
res1 = game1(positions)
res2 = max(game2(positions))

print(f"Answer 1: {res1}\nAnswer 2: {int(res2)}")

Answer 1: 739785
Answer 2: 444356092776315


In [12]:
# DAY22
import bisect

def parse(line):
    def parse_range(s):
        parts = s[2:].split("..")
        return (int(parts[0]), int(parts[1]))
    parts1 = line.split(" ")
    op = 1 if parts1[0] == "on" else 0
    p = [parse_range(p) for p in parts1[1].split(",")]
    return (op, p[0], p[1], p[2])

def intersects(cube1, cube2):
    return cube1[0][0] < cube2[0][1] and cube1[0][1] > cube2[0][0] and \
        cube1[1][0] < cube2[1][1] and cube1[1][1] > cube2[1][0] and \
        cube1[2][0] < cube2[2][1] and cube1[2][1] > cube2[2][0]

def get_lit_volume(ops, bound_region=None, log_progress=False):
    def get_coords_axis(ops, axis_id):
        c1 = [op[axis_id + 1][0] for op in ops]
        c2 = [op[axis_id + 1][1] + 1 for op in ops]
        return sorted(list(set(c1 + c2)))

    sections = [get_coords_axis(ops, i) for i in range(3)]
    grid = set()
    
    vol = 0
    for idx, (op, (x1, x2), (y1, y2), (z1, z2)) in enumerate(ops):
        if log_progress:
            print(f"op {idx} out of {len(ops)}, grid size: {len(grid)}")
        if bound_region != None:
            region = ((x1, x2), (y1, y2), (z1, z2))
            if not intersects(bound_region, region):
                continue
        i1 = bisect.bisect_left(sections[0], x1)
        j1 = bisect.bisect_left(sections[1], y1)
        k1 = bisect.bisect_left(sections[2], z1)
        k = k1
        while sections[2][k] <= z2:
            j = j1
            while sections[1][j] <= y2:
                i = i1
                while sections[0][i] <= x2:
                    pos = (i, j, k)
                    is_lit = pos in grid
                    if (op == 1) ^ is_lit:
                        dvol = (sections[0][i + 1] - sections[0][i]) * \
                            (sections[1][j + 1] - sections[1][j]) * \
                            (sections[2][k + 1] - sections[2][k])
                        if op == 1:
                            grid.add(pos)
                            vol += dvol
                        else:
                            grid.remove(pos)
                            vol -= dvol
                    i += 1
                j += 1
            k += 1
    return vol
                    
REGION0 = ((-50, 50), (-50, 50), (-50, 50))
ops = [parse(x.strip()) for x in open('../data/22.txt').readlines()]
res1 = get_lit_volume(ops, REGION0)
res2 = get_lit_volume(ops, None)

print(f"Answer 1: {res1}\nAnswer 2: {int(res2)}")

Answer 1: 577205
Answer 2: 1197308251666843


In [2]:
# DAY23
import heapq

HALL_ID = 4
HALL_LEN = 11
COSTS = [1, 10, 100, 1000]

def get_accessible_positions(pos):
    room_depth = (len(pos) - HALL_LEN) // 4
    room_pos_len = 4 * room_depth
    num_positions = room_pos_len + HALL_LEN

    def get_room_id(idx):
        nonlocal room_pos_len, num_positions, room_depth
        if idx >= room_pos_len and idx < num_positions:
            return HALL_ID
        elif idx < room_pos_len:
            return idx // room_depth
        else:
            return -1

    def get_unit_idx(c):
        if c in "abcd":
            return ord(c) - ord('a')
        elif c in "ABCD":
            return 4 + ord(c) - ord('A')
        else:
            return -1

    def get_hallway_exit_idx(room_id):
        nonlocal room_pos_len
        return room_pos_len + room_id * 2 + 2

    def is_hallway_exit(idx):
        nonlocal room_pos_len
        if idx < room_pos_len + 2 or idx > room_pos_len + 8:
            return False
        return (idx - room_pos_len - 2) % 2 == 0

    for i in range(num_positions):
        c = pos[i]
        if c == '.':
            continue
        room_id = get_room_id(i)
        unit_idx = get_unit_idx(c)
        if room_id != HALL_ID and not c.islower():
            blocked = False
            for j in range(i % room_depth):
                if pos[i - j - 1] != '.':
                    blocked = True
                    break
            if blocked:
                # blocked at the bottom of the room
                continue
                
            hallway_exit_idx = get_hallway_exit_idx(room_id)
                
            # move into hallway, to the left
            k = hallway_exit_idx - 1
            j = 1
            while k >= room_pos_len and pos[k] == '.':
                if not is_hallway_exit(k):
                    cost = COSTS[unit_idx % 4] * (i % room_depth + 1 + j)
                    pos1 = list(pos)
                    pos1[i] = '.'
                    pos1[k] = c.lower()
                    yield cost, "".join(pos1)
                k -= 1
                j += 1
            
            # move into hallway, to the right
            k = hallway_exit_idx + 1
            j = 1
            while k < num_positions and pos[k] == '.':
                if not is_hallway_exit(k):
                    cost = COSTS[unit_idx % 4] * (i % room_depth + 1 + j)
                    pos1 = list(pos)
                    pos1[i] = '.'
                    pos1[k] = c.lower()
                    yield cost, "".join(pos1)
                k += 1
                j += 1
        elif room_id == HALL_ID:
            # try move into our room
            room_id = unit_idx % 4
            offs_in_room = -1
            for j in range(room_depth):
                if pos[room_id * room_depth + j] == '.':
                    offs_in_room += 1
                else:
                    break
            if offs_in_room == -1:
                continue
                
            blocked = False
            for j in range(offs_in_room + 1, room_depth):
                rc = pos[room_id * room_depth + j]
                if rc != '.' and get_unit_idx(rc) % 4 != room_id:
                    blocked = True
                    break
            if blocked:
                # room still contains a unit of different type
                continue
                
            hallway_exit_idx = get_hallway_exit_idx(room_id)
            path_len = abs(i - hallway_exit_idx)
            if hallway_exit_idx > i and pos[i + 1:hallway_exit_idx + 1].count('.') != path_len:
                # path is blocked (going from left)
                continue
            if hallway_exit_idx < i and pos[hallway_exit_idx:i].count('.') != path_len:
                # path is blocked (going from right)
                continue
            cost = COSTS[unit_idx % 4] * (path_len + 1 + offs_in_room)
            pos1 = list(pos)
            pos1[i] = '.'
            pos1[room_id * room_depth + offs_in_room] = c
            yield cost, "".join(pos1)

def get_least_energy(pos):
    room_depth = (len(pos) - HALL_LEN) // 4
    goal = "".join(c * room_depth for c in 'ABCD')
    
    def is_goal(pos):
        nonlocal goal
        return goal == pos[:len(goal)].upper()
 
    to_explore = [(0, pos)]
    scores = {pos: 0}
    
    while len(to_explore) > 0:
        score, pos = heapq.heappop(to_explore)
        if is_goal(pos):
            return scores[pos]
        for cost, cpos in get_accessible_positions(pos):
            cscore = score + cost
            if cpos not in scores or scores[cpos] > cscore:
                scores[cpos] = cscore
                heapq.heappush(to_explore, (cscore, cpos))

def insert_extra(lines):
    return lines[:3] + ["  #D#C#B#A#", "  #D#B#A#C#"] + lines[3:]

def parse_pos(lines):
    room_depth = len(lines) - 3
    res = ""
    for j in range(4):
        for i in range(room_depth):
            res += lines[i + 2][3 + j*2]
    res += "..........."
    return res

lines = open('../data/23.txt').readlines()
start_pos1 = parse_pos(lines)
res1 = get_least_energy(start_pos1)
start_pos2 = parse_pos(insert_extra(lines))
res2 = get_least_energy(start_pos2)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 15358
Answer 2: 51436


In [4]:
# DAY24
def eval_prog(prog, inp):
    pos = 0
    reg = [0] * 4 # wxyz
    
    def get_val(v):
        nonlocal reg
        if v in "wxyz":
            return reg[ord(v) - ord('w')]
        return int(v)
                       
    for instr in prog:
        op = instr[0]
        if op == "inp":
            if pos == len(inp):
                return None
            reg[ord(instr[1]) - ord('w')] = int(inp[pos])
            pos += 1
        elif op == "add":
            p1 = ord(instr[1]) - ord('w')
            reg[p1] += get_val(instr[2])
        elif op == "mul":
            p1 = ord(instr[1]) - ord('w')
            reg[p1] *= get_val(instr[2])
        elif op == "div":
            p1 = ord(instr[1]) - ord('w')
            v2 = get_val(instr[2])
            if v2 == 0:
                return None
            reg[p1] //= v2
        elif op == "mod":
            p1 = ord(instr[1]) - ord('w')
            v1 = reg[p1]
            v2 = get_val(instr[2])
            if v1 < 0 or v2 <= 0:
                return None
            reg[p1] = v1 % v2
        elif op == "eql":
            p1 = ord(instr[1]) - ord('w')
            v1 = reg[p1]
            v2 = get_val(instr[2])
            reg[p1] = int(v1 == v2)
        else:
            return None
    return reg

MODEL_NUM_LEN = 14

"""
Extra constraints extracted by meticulousluy reading the input code:
n2 = 9 
n3 = 1
n5 = n4 - 2
n6 = n1 - 3
n10 = n9 + 5
n11 = n8 - 5
n12 = n7 + 4
n13 = n0 - 1
"""
ALLOWED_RANGES = [
    range(2, 10),
    range(4, 10),
    range(9, 10),
    range(1, 2),
    range(3, 10),
    (4, -2),
    (1, -3),
    range(1, 6),
    range(6, 10),
    range(1, 5),
    (9, 5),
    (8, -5),
    (7, 4),
    (0, -1),
]

def get_min_value(prog):
    res  = ""
    for r in ALLOWED_RANGES:
        if isinstance(r, tuple):
            (pos, d) = r
            c = chr(ord(res[pos]) + d)
        else:
            c = chr(ord('0') + min(x for x in r))
        res += c
    w, x, y, z = eval_prog(prog, res)
    assert z == 0, "Incorrect minimum value"
    return res
    
def get_max_value(prog):
    res  = ""
    for r in ALLOWED_RANGES:
        if isinstance(r, tuple):
            (pos, d) = r
            c = chr(ord(res[pos]) + d)
        else:
            c = chr(ord('0') + max(x for x in r))
        res += c
    w, x, y, z = eval_prog(prog, res)
    assert z == 0, "Incorrect maximum value"
    return res
            
prog = [x.strip().split(" ") for x in open('../data/24.txt').readlines()]
res1 = get_max_value(prog)
res2 = get_min_value(prog)

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 99919765949498
Answer 2: 24913111616151


In [5]:
# DAY25
def step(board, east):
    w, h = len(board[0]), len(board)
    j = 0
    res = [[' '] * w for _ in range(h)]
    moved = False
    for j in range(h):
        for i in range(w):
            c = board[j][i]
            if res[j][i] == ' ':
                res[j][i] = c
            if east and c == '>':
                i1 = (i + 1) % w
                if board[j][i1] == '.':
                    res[j][i1] = '>'
                    res[j][i] = '.'
                    moved = True
            elif not east and c == 'v':
                j1 = (j + 1) % h
                if board[j1][i] == '.':
                    res[j1][i] = 'v'
                    res[j][i] = '.'
                    moved = True
    return res, moved

board = [list(x.strip()) for x in open('../data/25.txt').readlines()]
steps = 0
while True:
    board, moved1 = step(board, True)
    board, moved2 = step(board, False)
    steps += 1
    if not moved1 and not moved2:
        break
        
print(f"Answer 1: {steps}")

Answer 1: 432
